In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from glob import glob
from concurrent.futures import ProcessPoolExecutor, as_completed

import numpy as np
import pandas as pd

import settings as conf
import metadata
from utils import is_number, chunker
from results.multixcan import MXPhenoInfo, MXPhenoResults

# fastENLOC reading functions

In [3]:
with open(conf.FASTENLOC_GTEX_TISSUES_FILE, 'r') as f:
    FASTENLOC_TISSUES_NAMES = set([x.strip() for x in f.readlines()])

In [4]:
# create summary of files
_path = os.path.join(conf.FASTENLOC_RESULTS_DIR['RapidGWASProject'], '**/*.enloc.enrich.out')
display(_path)
all_fastenloc_results_files = glob(_path)

'/mnt/phenomexcan_base/results/fastenloc/rapid_gwas_project/**/*.enloc.enrich.out'

In [5]:
len(all_fastenloc_results_files)

198401

In [6]:
PHENO_CODES = set(metadata.RAPID_GWAS_PHENO_INFO.index)

In [7]:
len(PHENO_CODES)

4359

In [8]:
phenos = []
tissues = []

for f in all_fastenloc_results_files:
    f = os.path.basename(f)
    split = f.split('-')
    
    tiss = split[-1].split('.')[0]
    if tiss not in FASTENLOC_TISSUES_NAMES:
        tiss = '-'.join(split[-2:]).split('.')[0]
    
    phen = split[1]
    if phen not in PHENO_CODES:
        phen = '-'.join(split[1:3])
    
    phenos.append(phen)
    tissues.append(tiss)

results_summary = pd.DataFrame({'pheno': phenos, 'tissue': tissues, 'file': all_fastenloc_results_files})

In [9]:
display(results_summary.shape)
assert results_summary.shape[0] == int(conf.FASTENLOC_EXPECTED_PHENOTYPES['RapidGWASProject'] * conf.GTEX_MODELS_N_EXPECTED_TISSUES)

(198401, 3)

In [10]:
results_summary.head()

,pheno,tissue,file
0,22617_9219,Thyroid,/mnt/phenomexcan_base/results/fastenloc/rapid_...
1,22617_9219,Nerve_Tibial,/mnt/phenomexcan_base/results/fastenloc/rapid_...
2,22617_9219,Brain_Hypothalamus,/mnt/phenomexcan_base/results/fastenloc/rapid_...
3,22617_9219,Small_Intestine_Terminal_Ileum,/mnt/phenomexcan_base/results/fastenloc/rapid_...
4,22617_9219,Breast_Mammary_Tissue,/mnt/phenomexcan_base/results/fastenloc/rapid_...


In [11]:
results_summary['pheno'].unique().shape

(4049,)

In [12]:
assert np.all([t in PHENO_CODES for t in results_summary['pheno'].unique()])

In [13]:
assert results_summary['tissue'].unique().shape[0] == conf.GTEX_MODELS_N_EXPECTED_TISSUES

In [14]:
assert np.all([t in FASTENLOC_TISSUES_NAMES for t in results_summary['tissue'].unique()])

In [15]:
_all_tissues_in_results = results_summary['tissue'].unique()
assert np.all([t in _all_tissues_in_results for t in FASTENLOC_TISSUES_NAMES])

In [16]:
assert len(all_fastenloc_results_files) == int(conf.SMULTIXCAN_EXPECTED_PHENOTYPES['RapidGWASProject'] * conf.GTEX_MODELS_N_EXPECTED_TISSUES)

In [86]:
def read_fe_enrich(filename):
    enrich_data = pd.read_csv(filename, sep='\s+', header=None)
    enrich_data = enrich_data.rename(columns={0: 'type', 1: 'alpha1', 2: 'se'})
    enrich_data = enrich_data[enrich_data['type'] == 'Enrichment']
    assert enrich_data.shape[0] == 1
    
    return enrich_data.iloc[0].loc[['alpha1', 'se']].astype(float)
    

def read_fe_enrich_pheno(pheno): 
    _tmp = results_summary[results_summary['pheno'] == pheno] 
    if _tmp.shape[0] == 0: 
        return None 
     
    _res = {}
#     _all_genes = set()
    for idx, _data in _tmp.iterrows():
        tissue_data = read_fe_enrich(_data.file)
        _res[_data.tissue] = tissue_data
#         _all_genes.update(tissue_data.index) 
    
    df = pd.DataFrame.from_dict(_res, orient='index')#.T#, index=_all_genes)#.fillna(-1).max(axis=1)
    df.index.rename('tissue', inplace=True)
    df = df.reset_index().assign(trait=pheno)
    return df[['tissue', 'trait', 'alpha1', 'se']]

In [87]:
results_summary.iloc[0].loc['file']

'/mnt/phenomexcan_base/results/fastenloc/rapid_gwas_project/22617_9219/fastenloc-22617_9219-Thyroid.enloc.enrich.out'

In [88]:
read_fe_enrich(results_summary.iloc[1].loc['file'])

alpha1     -0.713
se        593.007
Name: 1, dtype: float64

In [90]:
read_fe_enrich_pheno('22617_9219').head()

,tissue,trait,alpha1,se
0,Adipose_Subcutaneous,22617_9219,-6.094,1040.506
1,Adipose_Visceral_Omentum,22617_9219,5.625,217.299
2,Adrenal_Gland,22617_9219,4.155,395.173
3,Artery_Aorta,22617_9219,-10.846,2070.036
4,Artery_Coronary,22617_9219,-4.269,1800.014


### Testing

In [36]:
# testing
t = read_fe_enrich(os.path.join(conf.FASTENLOC_RESULTS_DIR['RapidGWASProject'], 'J15/fastenloc-J15-Whole_Blood.enloc.enrich.out'))

In [38]:
assert t['alpha1'] == -11.286
assert t['se'] == 764.307

In [91]:
t = read_fe_enrich_pheno('J15')

In [92]:
assert t.shape[0] == conf.GTEX_MODELS_N_EXPECTED_TISSUES

In [93]:
t.head()

,tissue,trait,alpha1,se
0,Adipose_Subcutaneous,J15,-12.109,995.299
1,Adipose_Visceral_Omentum,J15,-11.061,877.891
2,Adrenal_Gland,J15,-11.018,1024.396
3,Artery_Aorta,J15,-12.016,1029.679
4,Artery_Coronary,J15,-11.979,1336.767


In [102]:
_t_tmp = t[t['tissue'] == 'Adipose_Visceral_Omentum'].iloc[0]
assert _t_tmp['tissue'] == 'Adipose_Visceral_Omentum'
assert _t_tmp['trait'] == 'J15'
assert _t_tmp['alpha1'] == -11.061
assert _t_tmp['se'].round(3) == 877.891

_t_tmp = t[t['tissue'] == 'Artery_Coronary'].iloc[0]
assert _t_tmp['tissue'] == 'Artery_Coronary'
assert _t_tmp['trait'] == 'J15'
assert _t_tmp['alpha1'].round(3) == -11.979
assert _t_tmp['se'].round(3) == 1336.767

# Get Rapid GWAS phenotypes

In [103]:
all_smultixcan_files = glob(os.path.join(conf.SMULTIXCAN_RESULTS_DIR['RapidGWASProject'], '*.tsv.gz'))
all_smultixcan_phenotypes = [MXPhenoResults(p) for p in all_smultixcan_files]

In [104]:
assert len(all_smultixcan_files) == len(all_smultixcan_phenotypes) == conf.SMULTIXCAN_EXPECTED_PHENOTYPES['RapidGWASProject']

In [105]:
all_smultixcan_files[:5]

['/mnt/phenomexcan_base/results/smultixcan/rapid_gwas_project/smultixcan_20096_1_ccn30.tsv.gz',
 '/mnt/phenomexcan_base/results/smultixcan/rapid_gwas_project/smultixcan_2345_ccn30.tsv.gz',
 '/mnt/phenomexcan_base/results/smultixcan/rapid_gwas_project/smultixcan_N49_ccn30.tsv.gz',
 '/mnt/phenomexcan_base/results/smultixcan/rapid_gwas_project/smultixcan_100011_raw_ccn30.tsv.gz',
 '/mnt/phenomexcan_base/results/smultixcan/rapid_gwas_project/smultixcan_5221_ccn30.tsv.gz']

# Read all results

In [112]:
def _get_combined_results(phenos):
    all_res = [read_fe_enrich_pheno(pheno.pheno_info.pheno_code) for pheno in phenos]
    return pd.concat(all_res, ignore_index=True)

In [113]:
# testing
_tmp = _get_combined_results(all_smultixcan_phenotypes[:4])
assert _tmp.shape[0] == int(4 * 49)

In [116]:
_pending = read_fe_enrich_pheno('C_MYELOID-LEUKAEMIA')

In [117]:
_pending.head()

,tissue,trait,alpha1,se
0,Adipose_Subcutaneous,C_MYELOID-LEUKAEMIA,-11.482,1327.573
1,Adipose_Visceral_Omentum,C_MYELOID-LEUKAEMIA,-11.767,1643.732
2,Adrenal_Gland,C_MYELOID-LEUKAEMIA,-11.487,1752.318
3,Artery_Aorta,C_MYELOID-LEUKAEMIA,-11.046,1233.831
4,Artery_Coronary,C_MYELOID-LEUKAEMIA,-11.662,2048.158


In [120]:
def _run_all(phenotype_chunks, n_jobs=conf.N_JOBS_HIGH):
    all_results = []
    
    with ProcessPoolExecutor(max_workers=n_jobs) as executor:
        tasks = [executor.submit(_get_combined_results, chunk) for chunk in phenotype_chunks]
        for future in as_completed(tasks):
            res = future.result()
            all_results.append(res)

    return all_results

In [121]:
# phenotype_chunks = chunker(all_smultixcan_phenotypes[:5] + all_extra_phenotypes[:5], 2)
phenotype_chunks = chunker(all_smultixcan_phenotypes, 200)

In [122]:
all_results = _run_all(phenotype_chunks)

## Save as DataFrame

In [129]:
fastenloc_genes_associations = pd.concat(all_results, ignore_index=True)

display(fastenloc_genes_associations.shape)
display(fastenloc_genes_associations.head())

assert fastenloc_genes_associations.shape[0] == int(conf.FASTENLOC_EXPECTED_PHENOTYPES['RapidGWASProject'] * conf.GTEX_MODELS_N_EXPECTED_TISSUES)

(198401, 4)

,tissue,trait,alpha1,se
0,Adipose_Subcutaneous,O46,-12.018,1550.823
1,Adipose_Visceral_Omentum,O46,-11.835,1575.155
2,Adrenal_Gland,O46,-11.980,2014.509
3,Artery_Aorta,O46,-11.975,1679.678
4,Artery_Coronary,O46,-11.931,2131.642


In [130]:
assert fastenloc_genes_associations['tissue'].unique().shape[0] == conf.GTEX_MODELS_N_EXPECTED_TISSUES

In [131]:
assert fastenloc_genes_associations['trait'].unique().shape[0] == conf.FASTENLOC_EXPECTED_PHENOTYPES['RapidGWASProject']

In [133]:
assert fastenloc_genes_associations.shape == fastenloc_genes_associations.dropna().shape

In [136]:
# check columns data type
_tmp = fastenloc_genes_associations.dtypes.value_counts()
display(_tmp)
assert _tmp.shape[0] == 2

object     2
float64    2
dtype: int64

In [137]:
fastenloc_genes_associations.isna().sum().sum()

0

In [139]:
# some testing
t = fastenloc_genes_associations[fastenloc_genes_associations['trait'] == '22601_12253140']

In [140]:
t.head()

,tissue,trait,alpha1,se
2303,Adipose_Subcutaneous,22601_12253140,-12.415,722.794
2304,Adipose_Visceral_Omentum,22601_12253140,-12.281,776.066
2305,Adrenal_Gland,22601_12253140,-12.428,959.603
2306,Artery_Aorta,22601_12253140,-12.458,796.822
2307,Artery_Coronary,22601_12253140,-12.448,1042.841


In [143]:
tt = t[t['tissue'] == 'Brain_Frontal_Cortex_BA9'].iloc[0]
assert tt.loc['alpha1'].round(3) == -12.584
assert tt.loc['se'] == 1050.572

In [147]:
tt = t[t['tissue'] == 'Artery_Coronary'].iloc[0]
assert tt.loc['alpha1'].round(3) == -12.448
assert tt.loc['se'].round(3) == 1042.841

### Save

In [148]:
os.makedirs(conf.ANALYSES_DIR, exist_ok=True)

In [149]:
# Save
fastenloc_genes_associations_filename = os.path.join(conf.ANALYSES_DIR, f'fastenloc-alpha1-stats.pkl.xz')
display(fastenloc_genes_associations_filename)

'/mnt/phenomexcan_base/analyses/fastenloc-alpha1-stats.pkl.xz'

In [150]:
fastenloc_genes_associations.to_pickle(fastenloc_genes_associations_filename)

### Save in text format

In [151]:
output_file = os.path.join(conf.ANALYSES_DIR, 'fastenloc-alpha1-stats.tsv.gz')
display(output_file)

'/mnt/phenomexcan_base/analyses/fastenloc-alpha1-stats.tsv.gz'

In [153]:
fastenloc_genes_associations.to_csv(output_file, index=False, sep='\t', float_format='%.3f')

In [154]:
# test "for publication" file
_tmp = pd.read_csv(output_file, sep='\t')

In [155]:
display(_tmp.shape)
assert _tmp.shape == fastenloc_genes_associations.shape

(198401, 4)

In [156]:
_tmp.head()

,tissue,trait,alpha1,se
0,Adipose_Subcutaneous,O46,-12.018,1550.823
1,Adipose_Visceral_Omentum,O46,-11.835,1575.155
2,Adrenal_Gland,O46,-11.980,2014.509
3,Artery_Aorta,O46,-11.975,1679.678
4,Artery_Coronary,O46,-11.931,2131.642


In [157]:
assert not _tmp.isin([np.inf, -np.inf]).any().any()

In [159]:
assert not _tmp.isna().any().any()

In [161]:
assert fastenloc_genes_associations.equals(_tmp)